In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,10)

In [2]:
def read_csv(TRAIN_PATH):
    chunksize = 5_000_000 # batch size
    
    traintypes = {
        'fare': 'float32',
        'from': 'str',
        'to': 'str',
        'p_count': 'uint8', 
        'hour': 'uint8',
        'weekday': 'uint8',
        'week': 'uint8',
        'year': 'int16',
        'dist_e': 'float32',
        'dist_t': 'float32',
        'x0': 'float32',
        'x1': 'float32',
        'y0': 'float32',
        'y1': 'float32',
    }
    
    cols = list(traintypes.keys())
    
    # loading the dataframe into list of small dataframes
    df_list = []

    for df_chunk in pd.read_csv(TRAIN_PATH, usecols=cols, dtype=traintypes, chunksize=chunksize):
        df_list.append(df_chunk) 
        df = pd.concat(df_list)
        
    return df

In [3]:
train = read_csv('../input/train_cleaned_2.csv')

In [4]:
train.head()

,fare,from,to,hour,weekday,week,year,p_count,dist_e,dist_t,x0,x1,y0,y1
0,4.5,Queens,Queens,17,0,25,2009,1,0.638322,0.759079,-73.844315,-73.841614,40.721317,40.712276
1,16.9,Manhattan,Manhattan,16,1,1,2010,1,5.213395,6.717119,-74.016045,-73.979271,40.711304,40.782005
2,5.7,Manhattan,Manhattan,0,3,33,2011,2,0.852610,1.164240,-73.982735,-73.991241,40.761269,40.750561
3,7.7,Manhattan,Manhattan,4,5,16,2012,1,1.735600,1.943343,-73.987129,-73.991570,40.733143,40.758091
4,5.3,Manhattan,Manhattan,7,1,10,2010,1,1.228196,1.658726,-73.968094,-73.956657,40.768009,40.783764


In [5]:
y = train['fare']
X = train.drop(['fare'], axis=1)
del train

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.25, random_state=1)

del X,y

In [6]:
locations = [
    'Manhattan', 'JFK', 'LaGardia', 'Newark', 'Teterboro', 'Brooklin', 'Bronx', 'Staten', 'Queens', 'Westchester',
    'Nassau', 'Jersey', 'Bayonne', 'WStaton', 'West', 'WNewYork', 'North'        
]

# num_buckets = [0, num_buckets)

p_count = tf.feature_column.numeric_column('p_count')
taxicab = tf.feature_column.numeric_column('dist_t')
euclidean = tf.feature_column.numeric_column('dist_e')

x0 = tf.feature_column.numeric_column('x0')
x1 = tf.feature_column.numeric_column('x1')
y0 = tf.feature_column.numeric_column('y0')
y1 = tf.feature_column.numeric_column('y1')

numbuckets = 35
dx = np.linspace(-74.27, -72.98, numbuckets).tolist()
dy = np.linspace(40.56, 41.71, numbuckets).tolist()

x0_B = tf.feature_column.bucketized_column(x0, dx)
x1_B = tf.feature_column.bucketized_column(x1, dx)
y0_B = tf.feature_column.bucketized_column(y0, dy)
y1_B = tf.feature_column.bucketized_column(y1, dy)

xy_0 = tf.feature_column.crossed_column([x0_B, y0_B], numbuckets * numbuckets)
xy_1 = tf.feature_column.crossed_column([x1_B, y1_B], numbuckets * numbuckets)
xy_pair = tf.feature_column.crossed_column([xy_0, xy_1], numbuckets**4)

p_count_b = tf.feature_column.bucketized_column(p_count, [1,2,3])

# definition of the categorical columns
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
week = tf.feature_column.categorical_column_with_identity('week', num_buckets = 54)
year = tf.feature_column.categorical_column_with_vocabulary_list('year', [2009, 2010, 2011, 2012, 2013, 2014, 2015])

loc_from = tf.feature_column.categorical_column_with_vocabulary_list('from', locations)
loc_to = tf.feature_column.categorical_column_with_vocabulary_list('to', locations)

# definition of the feature crosses
from_to = tf.feature_column.crossed_column([loc_from, loc_to], len(locations)**2)
from_to_year = tf.feature_column.crossed_column([from_to, year], len(locations)**2 * 7)
from_to_year_week = tf.feature_column.crossed_column([from_to_year, week], len(locations)**2 * 7 * 54)
from_to_p_count = tf.feature_column.crossed_column([from_to, p_count_b], len(locations)**2 * 5)

hour_weekday = tf.feature_column.crossed_column([hour, weekday], 24 * 7)
weekday_week_year = tf.feature_column.crossed_column([weekday, week, year], 7 * 54 * 7)
week_year = tf.feature_column.crossed_column([week, year], 54 * 7)
weekday_year = tf.feature_column.crossed_column([weekday, year], 7 * 7)
hour_weekday_year = tf.feature_column.crossed_column([hour, weekday, year], 24 * 7 * 7)

In [7]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(from_to, 50),
    tf.feature_column.embedding_column(from_to_year, 50),
    tf.feature_column.embedding_column(from_to_year_week, 50),
    tf.feature_column.embedding_column(hour_weekday, 50),
    tf.feature_column.embedding_column(weekday_week_year, 50),
    tf.feature_column.embedding_column(week_year,50),
    tf.feature_column.embedding_column(weekday_year,50),
    tf.feature_column.embedding_column(hour_weekday_year, 50),
    tf.feature_column.embedding_column(xy_pair, 50),
    tf.feature_column.embedding_column(from_to_p_count, 50),
    

    # indicator columns
    tf.feature_column.indicator_column(loc_from),
    tf.feature_column.indicator_column(loc_to),
    tf.feature_column.indicator_column(hour),
    tf.feature_column.indicator_column(weekday),
    tf.feature_column.indicator_column(week),
    tf.feature_column.indicator_column(year),
    tf.feature_column.indicator_column(xy_0),
    tf.feature_column.indicator_column(xy_1),
    tf.feature_column.indicator_column(p_count_b),

    # numeric columns
    taxicab, euclidean, p_count
]

In [8]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [9]:
model_dir = './DNN_Regressor_regions_coords_overfit_crosses_r1'

file_writer = tf.summary.FileWriter(model_dir)

estimator = tf.estimator.DNNRegressor(
    model_dir = model_dir,
    feature_columns = deep_columns,
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001),
    hidden_units = [512,256,128,64,64,32,16], 
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './DNN_Regressor_regions_coords_overfit_crosses_r1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BF802DC470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(X_train, y_train, batch_size=512))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(X_eval, y_eval))

In [11]:
for i in range(5):
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./DNN_Regressor_regions_coords_overfit_crosses_r1\model.ckpt.
INFO:tensorflow:loss = 95626.47, step = 0
INFO:tensorflow:global_step/sec: 5.14874
INFO:tensorflow:loss = 110166.555, step = 100 (19.425 sec)
INFO:tensorflow:global_step/sec: 5.44016
INFO:tensorflow:loss = 100910.82, step = 200 (18.381 sec)
INFO:tensorflow:global_step/sec: 5.37469
INFO:tensorflow:loss = 103815.3, step = 300 (18.606 sec)
INFO:tensorflow:global

INFO:tensorflow:Saving checkpoints for 6450 into ./DNN_Regressor_regions_coords_overfit_crosses_r1\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-24-21:30:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords_overfit_crosses_r1\model.ckpt-6450
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-09-24-21:30:57
INFO:tensorflow:Saving dict for global step 6450: average_loss = 15.96162, global_step = 6450, label/mean = 1

INFO:tensorflow:loss = 7904.9927, step = 12300 (18.217 sec)
INFO:tensorflow:global_step/sec: 5.48666
INFO:tensorflow:loss = 15573.133, step = 12400 (18.226 sec)
INFO:tensorflow:global_step/sec: 5.48576
INFO:tensorflow:loss = 6066.593, step = 12500 (18.229 sec)
INFO:tensorflow:global_step/sec: 5.4942
INFO:tensorflow:loss = 4786.9365, step = 12600 (18.202 sec)
INFO:tensorflow:global_step/sec: 5.50535
INFO:tensorflow:loss = 4580.5996, step = 12700 (18.176 sec)
INFO:tensorflow:global_step/sec: 5.4669
INFO:tensorflow:loss = 3965.3352, step = 12800 (18.291 sec)
INFO:tensorflow:Saving checkpoints for 12825 into ./DNN_Regressor_regions_coords_overfit_crosses_r1\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:global_step/sec: 4.06336
INFO:tensorflow:loss = 6144.065, step = 12900 (24.599 sec)
INFO:tensorflow:global_step/sec: 5.48095
INFO:tensorflow:loss = 5346.871, step = 13000 (18.245 sec)
INFO:tensorflow:global_step/sec: 5.48144
INF

INFO:tensorflow:global_step/sec: 3.71772
INFO:tensorflow:loss = 6652.0786, step = 19000 (26.898 sec)
INFO:tensorflow:global_step/sec: 4.86248
INFO:tensorflow:loss = 5941.503, step = 19100 (20.566 sec)
INFO:tensorflow:global_step/sec: 4.85909
INFO:tensorflow:loss = 5912.42, step = 19200 (20.589 sec)
INFO:tensorflow:global_step/sec: 4.85135
INFO:tensorflow:loss = 5516.675, step = 19300 (20.604 sec)
INFO:tensorflow:global_step/sec: 4.86113
INFO:tensorflow:loss = 6379.686, step = 19400 (20.571 sec)
INFO:tensorflow:global_step/sec: 4.86022
INFO:tensorflow:loss = 14534.967, step = 19500 (20.575 sec)
INFO:tensorflow:global_step/sec: 4.86177
INFO:tensorflow:loss = 2935.7915, step = 19600 (20.569 sec)
INFO:tensorflow:global_step/sec: 4.8602
INFO:tensorflow:loss = 4003.2178, step = 19700 (20.575 sec)
INFO:tensorflow:global_step/sec: 4.86197
INFO:tensorflow:loss = 7035.985, step = 19800 (20.577 sec)
INFO:tensorflow:global_step/sec: 4.85462
INFO:tensorflow:loss = 9076.66, step = 19900 (20.590 sec)

INFO:tensorflow:global_step/sec: 5.44215
INFO:tensorflow:loss = 5114.52, step = 25800 (18.363 sec)
INFO:tensorflow:global_step/sec: 5.44978
INFO:tensorflow:loss = 5176.953, step = 25900 (18.346 sec)
INFO:tensorflow:global_step/sec: 5.48995
INFO:tensorflow:loss = 4191.6763, step = 26000 (18.215 sec)
INFO:tensorflow:global_step/sec: 5.46347
INFO:tensorflow:loss = 8980.439, step = 26100 (18.303 sec)
INFO:tensorflow:global_step/sec: 5.45236
INFO:tensorflow:loss = 4443.065, step = 26200 (18.341 sec)
INFO:tensorflow:global_step/sec: 5.46232
INFO:tensorflow:loss = 2949.8894, step = 26300 (18.307 sec)
INFO:tensorflow:global_step/sec: 5.46088
INFO:tensorflow:loss = 3848.6328, step = 26400 (18.328 sec)
INFO:tensorflow:global_step/sec: 5.4522
INFO:tensorflow:loss = 5531.5967, step = 26500 (18.329 sec)
INFO:tensorflow:global_step/sec: 5.44959
INFO:tensorflow:loss = 5298.0376, step = 26600 (18.363 sec)
INFO:tensorflow:global_step/sec: 5.4492
INFO:tensorflow:loss = 3939.666, step = 26700 (18.336 sec

...

INFO:tensorflow:global_step/sec: 5.63538
INFO:tensorflow:loss = 3746.5708, step = 161798 (17.761 sec)
INFO:tensorflow:global_step/sec: 5.43278
INFO:tensorflow:loss = 6743.3965, step = 161898 (18.394 sec)
INFO:tensorflow:global_step/sec: 5.50853
INFO:tensorflow:loss = 4813.6626, step = 161998 (18.151 sec)
INFO:tensorflow:global_step/sec: 5.64235
INFO:tensorflow:loss = 6731.781, step = 162098 (17.739 sec)
INFO:tensorflow:global_step/sec: 5.51788
INFO:tensorflow:loss = 3240.6404, step = 162198 (18.110 sec)
INFO:tensorflow:global_step/sec: 5.43328
INFO:tensorflow:loss = 6252.1235, step = 162298 (18.405 sec)
INFO:tensorflow:global_step/sec: 5.45092
INFO:tensorflow:loss = 6971.984, step = 162398 (18.347 sec)
INFO:tensorflow:global_step/sec: 5.4588
INFO:tensorflow:loss = 5624.441, step = 162498 (18.326 sec)
INFO:tensorflow:global_step/sec: 5.41436
INFO:tensorflow:loss = 6687.6973, step = 162598 (18.461 sec)
INFO:tensorflow:global_step/sec: 5.49655
INFO:tensorflow:loss = 6716.076, step = 16269

INFO:tensorflow:global_step/sec: 5.52418
INFO:tensorflow:loss = 4312.4375, step = 168498 (18.103 sec)
INFO:tensorflow:global_step/sec: 5.44244
INFO:tensorflow:loss = 20778.701, step = 168598 (18.373 sec)
INFO:tensorflow:global_step/sec: 5.47808
INFO:tensorflow:loss = 6241.572, step = 168698 (18.267 sec)
INFO:tensorflow:global_step/sec: 5.63955
INFO:tensorflow:loss = 9416.937, step = 168798 (17.719 sec)
INFO:tensorflow:global_step/sec: 5.63893
INFO:tensorflow:loss = 6635.838, step = 168898 (17.746 sec)
INFO:tensorflow:global_step/sec: 5.63422
INFO:tensorflow:loss = 5058.266, step = 168998 (17.733 sec)
INFO:tensorflow:global_step/sec: 5.64311
INFO:tensorflow:loss = 6142.099, step = 169098 (17.721 sec)
INFO:tensorflow:global_step/sec: 5.63383
INFO:tensorflow:loss = 8021.9277, step = 169198 (17.750 sec)
INFO:tensorflow:global_step/sec: 5.63072
INFO:tensorflow:loss = 3357.8743, step = 169298 (17.760 sec)
INFO:tensorflow:global_step/sec: 5.63853
INFO:tensorflow:loss = 7700.537, step = 169398

INFO:tensorflow:global_step/sec: 5.0239
INFO:tensorflow:loss = 5177.366, step = 174198 (19.920 sec)
INFO:tensorflow:global_step/sec: 5.01844
INFO:tensorflow:loss = 4125.6533, step = 174298 (19.911 sec)
INFO:tensorflow:global_step/sec: 5.02516
INFO:tensorflow:loss = 2700.3813, step = 174398 (19.903 sec)
INFO:tensorflow:global_step/sec: 5.0218
INFO:tensorflow:loss = 31421.826, step = 174498 (19.926 sec)
INFO:tensorflow:global_step/sec: 5.01761
INFO:tensorflow:loss = 4710.701, step = 174598 (19.914 sec)
INFO:tensorflow:global_step/sec: 5.04496
INFO:tensorflow:loss = 5127.1226, step = 174698 (19.837 sec)
INFO:tensorflow:global_step/sec: 5.03318
INFO:tensorflow:loss = 2733.6626, step = 174798 (19.868 sec)
INFO:tensorflow:global_step/sec: 5.02397
INFO:tensorflow:loss = 5357.591, step = 174898 (19.905 sec)
INFO:tensorflow:global_step/sec: 5.02391
INFO:tensorflow:loss = 7179.129, step = 174998 (19.905 sec)
INFO:tensorflow:global_step/sec: 5.01716
INFO:tensorflow:loss = 5961.6543, step = 175098

INFO:tensorflow:global_step/sec: 5.44005
INFO:tensorflow:loss = 5859.524, step = 180898 (18.381 sec)
INFO:tensorflow:global_step/sec: 5.52152
INFO:tensorflow:loss = 5513.737, step = 180998 (18.111 sec)
INFO:tensorflow:global_step/sec: 5.52918
INFO:tensorflow:loss = 3103.5344, step = 181098 (18.099 sec)
INFO:tensorflow:global_step/sec: 5.54022
INFO:tensorflow:loss = 3990.7734, step = 181198 (18.036 sec)
INFO:tensorflow:global_step/sec: 5.5656
INFO:tensorflow:loss = 3238.5747, step = 181298 (17.966 sec)
INFO:tensorflow:global_step/sec: 5.63645
INFO:tensorflow:loss = 4500.359, step = 181398 (17.742 sec)
INFO:tensorflow:global_step/sec: 5.54952
INFO:tensorflow:loss = 3002.921, step = 181498 (18.024 sec)
INFO:tensorflow:global_step/sec: 5.41849
INFO:tensorflow:loss = 3272.8872, step = 181598 (18.455 sec)
INFO:tensorflow:global_step/sec: 5.44637
INFO:tensorflow:loss = 4392.7817, step = 181698 (18.360 sec)
INFO:tensorflow:global_step/sec: 5.46442
INFO:tensorflow:loss = 4088.1594, step = 18179

KeyboardInterrupt: 

In [12]:
infer = pd.read_csv('../input/test_processed.csv', index_col='key')

generator = estimator.predict(input_fn=pred_input_fn(infer))
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

ids = infer.index
submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords_overfit_crosses_r1\model.ckpt-182635
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [13]:
submission.head()

,fare_amount
key,
2015-01-27 13:08:24.0000002,10.891527
2015-01-27 13:08:24.0000003,9.838905
2011-10-08 11:53:44.0000002,4.408840
2012-12-01 21:12:12.0000002,8.636699
2012-12-01 21:12:12.0000003,15.875583


In [14]:
submission.to_csv('../output/24.dnn_regressor.csv')